# **Requirements**







In [ ]:
!sudo apt install python3.5

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libpython3.5-minimal' for regex 'python3.5'
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [ ]:
!sudo pip3 install sumy
!sudo pip3 install git+git://github.com/miso-belica/sumy.git

     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 10.1MB 8.2MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746865 sha256=6a166b4306bf36c1dff9179731443481c91a88de13191e6cb79c08050b2b1b47
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21681 sha256=b8866424c5e62f404687b95d15430b7c8a94503bb3c48173257eded045112142
  Stored in directory: /root/.cache/pip/wheels/5a/4d/a1/510b12c5e65e0b2b3ce539b2af66da0fc57571e528924f4a52
Successfully built pycountry breadability
  Cloning git://github.com/miso-belica/sumy.git to /tmp/pip-req-build-ns27yfvp
  Running command git clone -q git://github.com/miso-belica/sumy.git /tmp/pip-req-build-ns27yfvp
  Created wheel for sumy: filename=sumy-0.8.1-py2.py3-none-any.whl size=87932 sha256=a7bab2d69ed77e82161d8e59f2a53d43c2edd1af66

In [ ]:
!python3 -c "import nltk; nltk.download('punkt')"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip3 install talon

     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 266kB 7.3MB/s 
  Created wheel for talon: filename=talon-1.4.4-cp36-none-any.whl size=35287 sha256=98f6c116f977ccc998420401c9201dc791902fc58016697637a3560bcfb9e40a
  Stored in directory: /root/.cache/pip/wheels/3f/ca/f9/748f5328839c03fe055f69110357b4718853dde8d872896209
Successfully built talon


# **Dataset**


*   The path of the datasetfile (txt-file of e-mails). --> optional
 




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **E-Mail-Summarization**


*   Modification of preprocessing-part of the Python-file email_summarization.py (Chauhan, 2018) from https://github.com/jatana-research/email-summarization
*   Using the 'Automatic Text Summarizer' from https://github.com/miso-belica/sumy




**Input**

In [ ]:
email_1 = '''
Hi Nicole,

Thank you for keeping me updated on this issue. I'm happy to hear that the issue got resolved after all. 
You can now use the app in its full functionality again. 
Also many thanks for your suggestions. We hope to improve this feature in the future. 

In case you experience any further problems with the app, please don't hesitate to contact me again.

Best regards,

John D.
Customer Support

0000 Sunshine Parkway
Mountain View, CA
United States'''

email_2 = '''
Dear Jennifer,

Thanks for the advertisement you sent me about the Fashion Show. I think it will be a fantastic event and I want to be there, but before I come to Paris I need to know a little more about train timetables and the show. Could you give me more information?
I’m trying to decide which train to take. I think the best option is to take the last one from Berlin, but it doesn’t arrive in Paris until about ten o’clock. Will that be OK, or is it too late for you?
I’ve never been to a fashion show before. I’ve never been to Paris, either, so I need your advice. What kind of clothes do you think I should wear? And what’s the weather like at the moment? Is it warm or rather cold? Do I need to bring some warm clothes? What about rain? What are the weather forecasts? Do they say it is going to rain within the next 2 days or not? I don’t know if I am able to pack into my bag, it isn’t too big. Maybe I will take 2 of them.
By the way, what shall we do on Sunday? How about going for a walk in the park, or going on a river cruise? Or maybe you’ve got some other, better ideas? Tell me if something comes to your mind.
Anyway, I can’t wait to see you. I’m looking forward to hearing from you. Answer me as soon as it’s possible.

Cheers,
Meghan''' 

email_3 = '''
Hello Michael,

Thank you for your e mail I received a week ago. I’m sorry that I didn’t reply to your letter sooner, but I’m spending my time now in a summer house where Internet connection is very weak.
Anyway, it’s a great place to relax and take a break after a hard year in our school.
I don’t watch TV here nor do I use a mobile phone or a computer, so I have much more time than on my hands than I do usually. I feel healthier and I’m tanned. I swim a lot in a local lake, ride a bike and play football with local people. I’ve made many new friends.
I wish you were here. Would you like to come along the next year? I bet you’ll be satisfied.
Answer me.

Thanks, 
P. G.'''


**Preprocessing**

In [ ]:
from talon.signature.bruteforce import extract_signature

def preprocess(email):
    """
    Performs preprocessing operations such as:
        1. Removing salutation line.
        2. Removing signature lines (only E-Mails in English are supported).
        3. Removing new line characters.
    """
    email, _ = extract_signature(email)
    
    lines = email.split('\n')

    for i in reversed(range(len(lines))):
        lines[i] = lines[i].strip()
        if lines[i] == '':
          lines.pop(i)
    email = '\n'.join(lines)
    email = extract_salutation(email)
    return email

def extract_salutation(email):

    lines = email.split('\n')

    for i in reversed(range(len(lines))):
        lines[i] = lines[i].strip()
        s = ['Hi','Hello','Hey','Dear']
        for j in range(len(s)):
          if s[j] in lines[i]:
            lines.pop(i)
    email = '\n'.join(lines)
    return email


**Process of Summarization**

In [ ]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

def summarizing(message):
  LANGUAGE = "english"
  SENTENCES_COUNT = 3


  if __name__ == "__main__":
    #for plain text files there exists two possibilities as following
    #parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
    parser = PlaintextParser.from_string(message, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)

    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)

**Output**

In [ ]:
#Preprocessed E-Mails
print("Message_1")
message_1 = preprocess(email_1)
print (message_1)
print(" ")

print("Message_2")
message_2 = preprocess(email_2)
print (message_2)
print(" ")

print("Message_3")
message_3 = preprocess(email_3)
print (message_3)
print(" ")

#Summarization of E-Mails
print("Summary_1")
summary_1 = summarizing(message_1)
print (summary_1)
print(" ")

print("Summary_2")
summary_2 = summarizing(message_2)
print (summary_2)
print(" ")

print("Summary_3")
summary_3 = summarizing(message_3)
print (summary_3)
print(" ")

Message_1
Thank you for keeping me updated on this issue. I'm happy to hear that the issue got resolved after all.
You can now use the app in its full functionality again.
Also many thanks for your suggestions. We hope to improve this feature in the future.
In case you experience any further problems with the app, please don't hesitate to contact me again.
 
Message_2
Thanks for the advertisement you sent me about the Fashion Show. I think it will be a fantastic event and I want to be there, but before I come to Paris I need to know a little more about train timetables and the show. Could you give me more information?
I’m trying to decide which train to take. I think the best option is to take the last one from Berlin, but it doesn’t arrive in Paris until about ten o’clock. Will that be OK, or is it too late for you?
I’ve never been to a fashion show before. I’ve never been to Paris, either, so I need your advice. What kind of clothes do you think I should wear? And what’s the weather 